In [1]:
import re
import json
import numpy as np
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt

In [2]:
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
df = pd.read_csv("../data/sentiment/train.csv")

In [4]:
mismatches = []

for row in df.itertuples():
    tokens = tokenizer(row.text)["input_ids"]

    l = 0
    res = ""
    start = row.text.index(row.important_span_text)
    end = start + len(row.important_span_text)
    decoded = []
    
    assert row.important_span_text in row.text
    
    for tok in tokens[1:-1]:
        dec = tokenizer.decode(tok)
        decoded.append(dec)

        
        if start - 1 <= l < end:
            res = res + dec
        
        l += len(dec)

    res = res[1:] if res.startswith(" ") else res

    assert l == len(row.text), decoded
    assert res == row.important_span_text

2022-04-29 23:40:50.412544: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-29 23:40:50.412566: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
